In [1]:
import os
import numpy as np
import pandas as pd
import pickle
from tqdm import tqdm
import math
import random
import gensim
from gensim import corpora

In [2]:
noun_path = '申告内容_plsa_8.csv'
noun_df = pd.read_csv(noun_path, encoding='UTF-8', header=0)
dependency_path = '問合理由_plsa_8.csv'
dependency_df = pd.read_csv(dependency_path, encoding='UTF-8', header=0)

In [4]:
Co_occurrence_matrix = pd.DataFrame(index=noun_df.columns[4:-2], columns=dependency_df.columns[4:-2]).fillna(0)

#可読性ZERO地獄4重ループ
for dependency_index, dependency_row in tqdm(dependency_df.iterrows()):
    for noun_index, noun_row in noun_df.iterrows():
        if dependency_row['行ID']-1 == noun_row['行ID']:
            for dependency_key, dependency_value in dependency_row[4:-2].iteritems():
                for noun_key, noun_value in noun_row[4:-2].iteritems():
                    if noun_value + dependency_value == 2:
                        Co_occurrence_matrix.at[noun_key, dependency_key] += 1

421it [00:18, 22.51it/s]


In [72]:
plsa = PLSA(Co_occurrence_matrix.values, 10)
plsa.train()

llh 0.8246408740607195
llh 0.8598018782906849
llh 0.9287521915160363
llh 1.0355757374421346
llh 1.1664553313538213
llh 1.268352556089663
llh 1.327723494636314
llh 1.3644759122400383
llh 1.386356524297673
llh 1.4012547630447227
llh 1.4136377301874314
llh 1.4259036542687988
llh 1.4379881273234203
llh 1.45158512470156
llh 1.4644120957724434
llh 1.4736088281306425
llh 1.4773367769240857
llh 1.4787798687677793
llh 1.4781188738216497
llh 1.4757318915731128
llh 1.4724366833067732
llh 1.4689847022446725
llh 1.46531464631104
llh 1.4617616848852675
llh 1.4595944501449811
llh 1.45891929585084
llh 1.4590897606813407
llh 1.45963062252705
llh 1.4602433422198542
llh 1.4607686652667569
llh 1.4611524945184557
llh 1.461420033554371
llh 1.4616577309686354
llh 1.4619409665422203
llh 1.4622531504729293
llh 1.4625340521342887
llh 1.4627605749211023
llh 1.462940435805988
llh 1.4630838761326106
llh 1.4631994240570947
llh 1.4632929373214754
llh 1.4633617897132014
llh 1.4633871496071167
llh 1.463324434287885
ll

In [78]:
# print ('P(z)')
# print (plsa.Pz)
# print ('P(x|z)')
# print (plsa.Px_z.shape)
# print ('P(y|z)')
# print (plsa.Py_z)

topics = ['word', 'topic1', 'topic2', 'topic3', 'topic4', 'topic5', 'topic6', 'topic7', 'topic8', 'topic9', 'topic10']
x_matrix = pd.DataFrame(index=[], columns=topics)
y_matrix = pd.DataFrame(index=[], columns=topics)

# print ('P(z|x)')
Pz_x = plsa.Px_z.T * plsa.Pz[None, :]
# Pz_x = Pz_x / np.sum(Pz_x, axis=1)[:, None]

for i, x in enumerate(Co_occurrence_matrix.index):
    # x_row = pd.Series([x, Pz_x[i][0], Pz_x[i][1], Pz_x[i][2], Pz_x[i][3], Pz_x[i][4], Pz_x[i][5], Pz_x[i][6], Pz_x[i][7], Pz_x[i][8], Pz_x[i][9]], index=x_matrix.columns)
    x_row = pd.Series([x, plsa.Px_z.T[i][0], plsa.Px_z.T[i][1], plsa.Px_z.T[i][2], plsa.Px_z.T[i][3], plsa.Px_z.T[i][4], plsa.Px_z.T[i][5], plsa.Px_z.T[i][6], plsa.Px_z.T[i][7], plsa.Px_z.T[i][8], plsa.Px_z.T[i][9]], index=x_matrix.columns)
    x_matrix = x_matrix.append(x_row, ignore_index=True)

# print ('P(z|y)')
Pz_y = plsa.Py_z.T * plsa.Pz[None, :]
# Pz_y = Pz_y / np.sum(Pz_y, axis=1)[:, None]
for i, y in enumerate(Co_occurrence_matrix.columns):
    # y_row = pd.Series([y, Pz_y[i][0], Pz_y[i][1], Pz_y[i][2], Pz_y[i][3], Pz_y[i][4], Pz_y[i][5], Pz_y[i][6], Pz_y[i][7], Pz_y[i][8], Pz_y[i][9]], index=y_matrix.columns)
    y_row = pd.Series([y, plsa.Py_z.T[i][0], plsa.Py_z.T[i][1], plsa.Py_z.T[i][2], plsa.Py_z.T[i][3], plsa.Py_z.T[i][4], plsa.Py_z.T[i][5], plsa.Py_z.T[i][6], plsa.Py_z.T[i][7], plsa.Py_z.T[i][8], plsa.Py_z.T[i][9]], index=y_matrix.columns)
    y_matrix = y_matrix.append(y_row, ignore_index=True)

# x_matrix.to_csv('申告内容_plsa_result_pxz.csv')
# y_matrix.to_csv('問合理由_plsa_result_pyz.csv')

In [79]:
Pdx = 1/noun_df.iloc[:,4:-2]
Pdy = 1/dependency_df.iloc[:,4:-2]
Pdx[np.isinf(Pdx)] = 0
Pdy[np.isinf(Pdy)] = 0
Pdxz = np.dot(Pdx.values, plsa.Px_z.T)
Pdyz = np.dot(Pdy.values, plsa.Py_z.T)
Pddx = 0.5
Pddy = 0.5
Pdz = (Pddx * Pdxz) + (Pddy * Pdyz)
Pd = np.dot(Pdz, plsa.Pz)
score = Pdz.T/Pd
score[np.isnan(score)] = 0
df = pd.DataFrame(score.T)
df['原文'] = noun_df['原文']
# df.to_csv('result.csv')

In [97]:
score.shape
# Pdyz.shape

(10, 421)